## Accessing Socrata Open Data API
[Notebook from: http://bit.ly/byte-size-data-science]<br/>### discovery

The documentation is found at: https://socratadiscovery.docs.apiary.io/#reference

Socrates has a discovery API over HTTP requests. We can find data in multiple ways:

- by id: each asset has a unique 9 character identifier
- by domain: a specific "repository" such as "data.oregon.org"
- by category or tag: assets may be labeled with zero or more categories and zero or more tags
- by type: api, calendar, chart, datalens, dataset, federated_href, file, filter, form, href, link, map, measure, story, visualization
- by domain-specific metadata
- by attribution
- by license
- by query term: any text found in name, description, category, tags, column names, column fieldnames, column descriptions, attribution
- by parent ID
- by provenance
- by owner
- by granted shares
- by column name
- by visibility
- by public/private asset
- . . .

In [ ]:
# Libraries needed in the notebook
import urllib3, requests, json
import pandas as pd

# pd.set_option('display.max_colwidth', -1)

## Domains
There are two URLs used to discover domains:
- for Americas: http://api.us.socrata.com/api/catalog/v1 (Canada, Columbia, Mexico, USA, . . .)
- for Europe: http://api.eu.socrata.com/api/catalog/v1 (Italy, Netherland, Spain, UK, Others)

Let's get a list of domains that has assets and the count of assets in each domain.

**Note that not all assets are datasets**

### Finding domains in Americas

In [ ]:
url_us="http://api.us.socrata.com/api/catalog/v1"
urlDomains = url_us + "/domains"
response = requests.get(urlDomains)
jsondoc = json.loads(response.text)
domains_us_df = pd.io.json.json_normalize(jsondoc['results'])
# Number of entries/number of domains
print('Number of domains: ' + str(domains_us_df.shape[0]))
print('Number of assets: ' + str(domains_us_df['count'].agg('sum')))

In [ ]:
# Top 15 domains
domains_us_df.sort_values('count', ascending=False).head(15)

### Finding domains from the European URL

In [ ]:
url_eu = "http://api.eu.socrata.com/api/catalog/v1" 
urlDomains = url_eu + "/domains"
response = requests.get(urlDomains)
jsondoc = json.loads(response.text)
domains_eu_df = pd.io.json.json_normalize(jsondoc['results'])
# Number of entries/number of domains
print('Number of domains: ' + str(domains_eu_df.shape[0]))
print('Number of assets: ' + str(domains_eu_df['count'].agg('sum')))

In [ ]:
# Only 26 domains (right now), list them all
domains_eu_df.sort_values('count', ascending=False).head(26)

## Searching by type - datasets
We can select assets by types using the `only` parameter: api, calendar, chart, datalens, dataset, federated_href, file, filter, form, href, link, map, measure, story, visualization

We're interested in datasets.

We can also search by categories Here are some available categories:<br/>
demographics, economy, education, environment, finance, health, housing & development, infrastructure, politics, public safety, recreation, transportation 


In [ ]:
# By default, it returns 100 records.
# We can get more by using pagination parameters: offset and limit (up to 10000 records)
# If there are more, we have to use the scroll_id parameter
# the ID of the last result in the previously fetched chunk of results. 

url="http://api.us.socrata.com/api/catalog/v1"
# Retrieve only datasets
urldatasets = url + "?only=dataset"
offset=0
limit=10000
limit2=100
done = 0
scroll_id=""
all_records = dict(results=[])
while (done == 0) :
    page = "&scroll_id=" + scroll_id + "&limit=" + str(limit)
    # print(url + page)
    response = requests.get(urldatasets + page)
    if response.status_code != 200 :
        print(response.status_code)
        done = 1
        break
    if (offset == 0) :
        jsondoc = json.loads(response.text)
        scroll_id = jsondoc['results'][len(jsondoc['results']) - 1]['resource']['id']
        max_records = jsondoc['resultSetSize']
        for val in jsondoc['results'] :
            all_records['results'].append(val)
    else :
        jsondoc = json.loads(response.text)
        scroll_id = jsondoc['results'][len(jsondoc['results']) - 1]['resource']['id']
        for val in jsondoc['results'] :
            all_records['results'].append(val)

    offset += limit
    if (offset >= max_records) :
        done = 1

catalog_df = pd.io.json.json_normalize(all_records['results'])
catalog_df.shape

In [ ]:
catalog_df.head()

### List domains with the most datasets
You can find the meaning of the domain suffix at: https://en.wikipedia.org/wiki/List_of_Internet_top-level_domains

Some suffixes:
- .au : Australia
- .ca : Canada
- .co : Columbia
- .com: Commercial entities
- .edu: Education
- .gov: US national and state
- .org: Possibly non-profit but open (some cities use it)
- .us : US cities?


In [ ]:
# List the top domains and the number of datasets available
# you can find the domain name postfix meaning at:
# https://en.wikipedia.org/wiki/List_of_Internet_top-level_domains

print("Number of domains: " + str(catalog_df['metadata.domain'].nunique()))

catalog_df.groupby(['metadata.domain']).agg(['count'])['link'].\
           sort_values('count',ascending=False).reset_index(drop=False).head(30)

In [ ]:
# What about the bottom 30 domains?
catalog_df.groupby(['metadata.domain']).agg(['count'])['link'].\
           sort_values('count',ascending=True).reset_index(drop=False).head(30)

### Count for the European URL

In [ ]:
# By default, it returns 100 records.
# We can get more by using pagination parameters: offset and limit (up to 10000 records)
# If there are more, we have to use the scroll_id parameter
# the ID of the last result in the previously fetched chunk of results. 

url_eu = "http://api.eu.socrata.com/api/catalog/v1"
# Retrieve only datasets
urldatasets = url_eu + "?only=dataset"
offset=0
limit=10000
limit2=100
done = 0
scroll_id=""
all_records = dict(results=[])
while (done == 0) :
    page = "&scroll_id=" + scroll_id + "&limit=" + str(limit)
    # print(url + page)
    response = requests.get(urldatasets + page)
    if response.status_code != 200 :
        print(response.status_code)
        done = 1
        break
    if (offset == 0) :
        jsondoc = json.loads(response.text)
        scroll_id = jsondoc['results'][len(jsondoc['results']) - 1]['resource']['id']
        max_records = jsondoc['resultSetSize']
        for val in jsondoc['results'] :
            all_records['results'].append(val)
    else :
        jsondoc = json.loads(response.text)
        scroll_id = jsondoc['results'][len(jsondoc['results']) - 1]['resource']['id']
        for val in jsondoc['results'] :
            all_records['results'].append(val)

    offset += limit
    if (offset >= max_records) :
        done = 1

catalog_eu_df = pd.io.json.json_normalize(all_records['results'])
catalog_eu_df.shape

## Back to Americas
We need to explore the different categories. The first thig to note is that many records don't have entries in each attribute.

`classification.categories`, `classification.domain_category`, `classification.domain_metadata`, `classification.domain_tags`, `classification.tags`

### classification.categories

In [ ]:
# How many records include classification.categories?
# We find out by looking at which record does not have an empty list
print('classification.categories: ' + 
      str(catalog_df[catalog_df['classification.categories'].map(len) > 0]['classification.categories'].count()) )

In [ ]:
# We see that several entries don't have categories.
# List the categories found in the datasets listed
my_dict = dict()
for classification in catalog_df['classification.categories'] :
    for val in classification :
        if val in my_dict :
            my_dict[val] += 1
        else :
            my_dict[val] = 1
my_dict

### classification.domain_category

In [ ]:
# This columns seems to be string or float when empty
import operator
from heapq import nlargest
from operator import itemgetter

my_dict2 = dict()
for val in catalog_df['classification.domain_category'] :
    if (isinstance(val, float) == False) :
        if val in my_dict2 :
            my_dict2[val] += 1
        else :
            my_dict2[val] = 1
print("Number of entries: " + str(len(my_dict2)))
print("")
# sorted_my_dict2 = sorted(my_dict.items(), key=operator.itemgetter(1))
for name, score in nlargest(20, my_dict2.items(), key=itemgetter(1)):
    print (name, score)

### classification.domain_metadata
The meatadata section is a list of dictionaries. For example:

`{'key': 'Update_Update-Frequency', 'value': 'Monthly'}`<br/>
`{'key': 'Dataset-Summary_Granularity', 'value': ''}`

It is worth explring the metadata available to search on the characteristics of the daatasets

In [ ]:
# Number of entries with metadata
print('classification.domain_metadata: ' + 
      str(catalog_df[catalog_df['classification.domain_metadata'].map(len) > 0]['classification.domain_metadata'].count() ))

In [ ]:
# List of dictionaries
my_dict3 = dict()
for metadata in catalog_df['classification.domain_metadata'] :
    for val in metadata :
        if val['key'] in my_dict3 :
            my_dict3[val['key']] += 1
        else :
            my_dict3[val['key']] = 1
print("Number of entries: " + str(len(my_dict2)))
print("")
# sorted_my_dict3 = sorted(my_dict3.items(), key=operator.itemgetter(1))
for name, score in nlargest(20, my_dict3.items(), key=itemgetter(1)):
    print (name, score)

### classification.domain_tags
Tags can be used to limit the number of datasets we want to explore.

In [ ]:
# Number of domain_tags
print('classification.domain_tags: ' +
      str(catalog_df[catalog_df['classification.domain_tags'].map(len) > 0]['classification.domain_tags'].count() ))

In [ ]:
my_dict4 = dict()
for tags in catalog_df['classification.domain_tags'] :
    for val in tags :
        if val in my_dict4 :
            my_dict4[val] += 1
        else :
            my_dict4[val] = 1
print("Number of entries: " + str(len(my_dict4)))
print("")
# sorted_my_dict4 = sorted(my_dict.items(), key=operator.itemgetter(1))
for name, score in nlargest(20, my_dict4.items(), key=itemgetter(1)):
    print (name, score)

### classification.tags
This field appears to be mostly empty

In [ ]:
print('classification.tags: ' +
      str(catalog_df[catalog_df['classification.tags'].map(len) > 0]['classification.tags'].count() ))

In [ ]:
my_dict5 = dict()
for tags in catalog_df['classification.tags'] :
    for val in tags :
        if val in my_dict5 :
            my_dict5[val] += 1
        else :
            my_dict5[val] = 1
print("Number of entries: " + str(len(my_dict5)))
print("")
# sorted_my_dict5 = sorted(my_dict.items(), key=operator.itemgetter(1))
for name, score in nlargest(20, my_dict5.items(), key=itemgetter(1)):
    print (name, score)

In [ ]:
# Let's see the records that use tags
catalog_df[catalog_df['classification.tags'].map(len) > 0][
    ['classification.tags','resource.name','resource.description','metadata.domain','resource.id']].head(20)

### Number of non-null values per attributes
We start with a simple count. This is fine for some attributes but some attributes are list or dictionaries. This requires looking also at sizes.

In [ ]:
catalog_df.count()

## Other fields
There are multiple other fields that could be used to limit the search. For example:
- metadata.domain
- resource.columns_name
- resource.createdAt
- resource.name
- resource.description
- resource.updatedAt

Of those, the domain name and updatedAt attributes should be the most useful.

Other fields such as page view statistics could potentially be useful.

## Timestamp
An important search criterion is a date such as date updated.<br/>
This way we can ask for datasets updated at least in the last 3 months for example.

For that, we have to make sure the date fields have the proper data type.

In [ ]:
catalog_df.dtypes

In [ ]:
# Convert the date field to timestamps
catalog_df['resource.createdAt'] = \
           catalog_df['resource.createdAt'].apply(pd.to_datetime, infer_datetime_format=True, errors='coerce')
catalog_df['resource.updatedAt'] = \
           catalog_df['resource.updatedAt'].apply(pd.to_datetime, infer_datetime_format=True, errors='coerce')
catalog_df[['resource.name','metadata.domain','resource.id','resource.createdAt',
            'resource.updatedAt','resource.description','link','resource.columns_field_name', 
            'resource.columns_datatype']].head()

In [ ]:
catalog_df[['resource.createdAt','resource.updatedAt']].dtypes

In [ ]:
# Look for chicago datasets that talk about crashes
catalog_df[catalog_df['metadata.domain'] == 'data.cityofchicago.org'][
    catalog_df['resource.description'].str.contains("[Cc]rash") == True][
    ['resource.name','resource.id','resource.description']].head(50)

## Save the dataset
We can keep retrieving the dataset from Socrata. It makes sense since this way we get all the updates.<br/>
There is a limit on the number of queries and volume of data that can be retrieve so avoiding unnecessary queries makes sense.

We can work with a fix list and work from that for a while. We can always re-create the list regularly.

In [ ]:
# Don't write out the Pandas DataFrame index
catalog_df.to_csv("./SocrataCatalog.csv", index=False)

In [ ]:
%ls -l

## Write to COS
Write the local file to the Cloud Object Storage

**Your credentials will be different from the ones listed here**. 

In [ ]:
from botocore.client import Config
import ibm_boto3

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials = {
    'IBM_API_KEY_ID': '9OUS8qWWwiqFtRkeH6njgVaar',
    'IAM_SERVICE_ID': 'iam-ServiceI4f37-abd9-38b10084d177',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.bluemix.net/oidc/token',
    'BUCKET': 'pr-paqxy5fmsmaykn'
}
cos = ibm_boto3.client(service_name='s3',
                       ibm_api_key_id=credentials["IBM_API_KEY_ID"],
                       ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
                       config=Config(signature_version='oauth'),
        endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

Bucket=credentials["BUCKET"]

cos.upload_file('./SocrataCatalog.csv', Bucket, 'SocrataCatalog.csv')